In [6]:
import pandas as pd
reads_matrix = pd.read_hdf('all_reads_matrix.h5', key='matrix')

In [69]:
organisms = pd.read_csv('all_reads_matrix_organisms.csv', index_col='sample_id')

In [17]:
import torch

In [60]:
model = torch.nn.Sequential(
    torch.nn.Linear(15501, 8192),
    torch.nn.Dropout(.1),
    torch.nn.ReLU(),
    torch.nn.Linear(8192, 4096),
    torch.nn.Dropout(.1),
    torch.nn.ReLU(),
    torch.nn.Linear(4096, 2048),
    torch.nn.Dropout(.1),
    torch.nn.ReLU(),
    torch.nn.Linear(2048, 1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),    
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 1),
    torch.nn.Sigmoid()
)

In [61]:
criterion = torch.nn.BCELoss()

In [62]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [82]:
import numpy as np
class HMDataSet(torch.utils.data.Dataset):
    def __init__(self, reads_matrix, organisms):
        self.reads_matrix = reads_matrix
        self.organisms = organisms
        
        omap = { 'Mus musculus': 0, 'Homo sapiens': 1}
        self.ocats = organisms.organism.map(omap).values
    
    def __len__(self):
        return self.organisms.shape[0]
    
    def __getitem__(self, idx):
        return self.reads_matrix.iloc[idx].values, self.ocats[idx]

In [83]:
full_ds = HMDataSet(reads_matrix, organisms)

In [88]:
train_size = int(0.8 * len(full_ds))
test_size = len(full_ds) - train_size
train_ds, test_ds = torch.utils.data.random_split(full_ds, [train_size, test_size])

In [99]:
params = {
   'batch_size': 1000,
   'shuffle': True,
   'num_workers': 1
}

In [100]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [101]:
train_generator = torch.utils.data.DataLoader(train_ds, **params)
test_generator = torch.utils.data.DataLoader(test_ds, **params)

In [102]:

n_epochs = 10
loss_history = []
for i in range(n_epochs):
    def closure():
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, Y)
        loss.backward()
        return loss
    
    # Training
    for local_batch, local_labels in train_generator:
        print(local_batch, local_labels)
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
    
    #loss = optimizer.step(closure)
    #print(f'Epoch: [{i+1}/{n_epochs}], Loss: {loss:.4f}')
    #loss_history.append(loss)

BrokenPipeError: [Errno 32] Broken pipe

In [58]:
loss

tensor(0.6722, grad_fn=<BinaryCrossEntropyBackward>)

In [52]:
loss

tensor(0.6941, grad_fn=<BinaryCrossEntropyBackward>)